# Lab 12-1 hello-RNN

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices()

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Lab 12 RNN
import numpy as np
import tensorflow as tf

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
# x_data = [[0, 1, 0, 2, 3, 3]]  # hihell
y_data = [[1, 0, 2, 3, 3, 4]]  # ihello

num_classes = 5
input_dim = 5  # one-hot size, same as hidden_size to directly predict one-hot
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

x_one_hot = np.array([[[1, 0, 0, 0, 0],    # h 0
                       [0, 1, 0, 0, 0],    # i 1
                       [1, 0, 0, 0, 0],    # h 0
                       [0, 0, 1, 0, 0],    # e 2
                       [0, 0, 0, 1, 0],    # l 3
                       [0, 0, 0, 1, 0]]],  # l 3
                     dtype=np.float32)

# change result to one-hot encoding
y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)
print(x_one_hot.shape)
print(y_one_hot.shape)

(1, 6, 5)
(1, 6, 5)


In [5]:
tf.model = tf.keras.Sequential()

# make cell and add it to RNN layer
# input_shape = (1,6,5) => number of sequence (batch), length of sequence, size of input dim
cell = tf.keras.layers.LSTMCell(units=num_classes, input_shape=(sequence_length, input_dim))
# return_sequences : Boolean (default False). Whether to return the last output in the output sequence, or the full sequence.
tf.model.add(tf.keras.layers.RNN(cell=cell, return_sequences=True))

# single LSTM layer can be used as well instead of creating LSTMCell
# tf.model.add(tf.keras.layers.LSTM(units=num_classes, input_shape=(sequence_length, input_dim), return_sequences=True))

# fully connected layer
# tf.keras.layers.TimeDistributed : This wrapper allows to apply a layer to every temporal slice of an input.
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))

tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

# train
tf.model.fit(x_one_hot, y_one_hot, epochs=50)
tf.model.summary()

Epoch 1/50


2022-08-26 16:12:32.756947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 908ms/step - loss: 1.6075 - accuracy: 0.1667
Epoch 2/50
1/1 [==============================] - 0s 50ms/step - loss: 1.4821 - accuracy: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 49ms/step - loss: 1.4286 - accuracy: 0.3333
Epoch 4/50
1/1 [==============================] - 0s 51ms/step - loss: 1.3431 - accuracy: 0.3333
Epoch 5/50
1/1 [==============================] - 0s 49ms/step - loss: 1.2614 - accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 49ms/step - loss: 1.1444 - accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 50ms/step - loss: 1.0018 - accuracy: 0.6667
Epoch 8/50
1/1 [==============================] - 0s 49ms/step - loss: 0.8910 - accuracy: 0.6667
Epoch 9/50
1/1 [==============================] - 0s 50ms/step - loss: 0.7802 - accuracy: 0.8333
Epoch 10/50
1/1 [==============================] - 0s 49ms/step - loss: 0.6848 - accuracy: 0.8333
Epoch 11/50
1/1 [======================

In [6]:
predictions = tf.model.predict(x_one_hot)

# for문 한 번만 사용 => batch_size == 1 (one sentence)
for i, prediction in enumerate(predictions):
    print(prediction)
    # print char using argmax, dict
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print("\tPrediction str: ", ''.join(result_str))

1/1 [==============================] - 0s 103ms/step
[[2.8132971e-03 9.9716610e-01 1.7708699e-05 3.1769025e-07 2.5019294e-06]
 [9.9498087e-01 3.0229976e-03 1.5909565e-03 4.0348584e-04 1.8271716e-06]
 [7.0171466e-04 2.9461489e-05 9.9826950e-01 9.3516917e-04 6.4244537e-05]
 [2.4537293e-03 1.0573438e-08 7.0109287e-05 9.9745411e-01 2.2024898e-05]
 [2.8210357e-04 1.7210725e-08 3.2953027e-05 9.9892932e-01 7.5556873e-04]
 [6.4142444e-07 1.5163514e-05 1.2194988e-06 2.7260422e-03 9.9725705e-01]]
	Prediction str:  ihello


2022-08-26 16:12:35.534303: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
